## Welcome to my kernel on Meta Kaggle Dataset

Finally I understood that Meta-Kaggle is a dataset; <br>I ever saw amazing kernels that the source was <b>[Dataset no longer available]</b> and only now I became to here. 

# Objective:
I will explore this interesting that talk about us =D

- How many users are registered on Kaggle?
- How many of them are actively on the platform?
- What's the distribution of the tiers?
- When the plataform reached the mark of 1 million registrations?
- Which are the mean average rate of registrations by day?
- Which was the peak of registrations?
- How long time in mean takes to get the Tiers?

And many more interesting questions that probably will raise

## <font color="red"> I'm near of grandmaster tier, so, if you find this kernel useful or interesting, please don't forget to upvote the kernel =)</font>

# Importing Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Util Functions

In [ ]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    for name in summary['Name'].value_counts().index:
        summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary

def cross_heatmap(df, cols, normalize=False, values=None, aggfunc=None):
    temp = cols
    cm = sns.light_palette("green", as_cmap=True)
    return (round(pd.crosstab(df[temp[0]], df[temp[1]], 
                       normalize=normalize, values=values, aggfunc=aggfunc) * 100,2)).style.background_gradient(cmap = cm)

def date_time_col(df, col, form='%m/%d/%Y'):
    df[col] = pd.to_datetime(df[col], format=form)
    return df[col]

## Importing the dataset that we will work at. 

In [ ]:
kernel_tags = pd.read_csv('/kaggle/input/meta-kaggle/KernelTags.csv')
kernels = pd.read_csv('/kaggle/input/meta-kaggle/Kernels.csv')
kernel_versions = pd.read_csv('/kaggle/input/meta-kaggle/KernelVersions.csv')
kernel_votes = pd.read_csv('/kaggle/input/meta-kaggle/KernelVotes.csv')
kernel_langs = pd.read_csv('/kaggle/input/meta-kaggle/KernelLanguages.csv')
follow_users = pd.read_csv('/kaggle/input/meta-kaggle/UserFollowers.csv')
users_achi = pd.read_csv("/kaggle/input/meta-kaggle/UserAchievements.csv")
users = pd.read_csv('/kaggle/input/meta-kaggle/Users.csv')
tags = pd.read_csv('/kaggle/input/meta-kaggle/Tags.csv')

comp = pd.read_csv('/kaggle/input/meta-kaggle/Competitions.csv')

In [ ]:
kernel_tags.rename(columns={'Id':'KernelId'}, inplace=True)
kernels.rename(columns={'Id':'KernelId'}, inplace=True)
kernel_versions.rename(columns={'Id':'KVersId', 
                                'TotalVotes':'TotalVotes_version'}, inplace=True)
kernel_votes.rename(columns={'Id':'KernVotId'}, inplace=True)
kernel_langs.rename(columns={'Id':'KernelLanguageId'}, inplace=True)
follow_users.rename(columns={'Id':'UserId'}, inplace=True)
users_achi.rename(columns={'Id':'AchId'}, inplace=True)
users.rename(columns={'Id':'UserId'}, inplace=True)
users_achi.rename(columns={'Id':'AchId'}, inplace=True)
#dataset.rename(columns={'Id':'dsetId'}, inplace=True)
kernel_versions.rename(columns={'TotalVotes':'TotalVotes_version'}, inplace=True)

Let's start with an overview of the following tables:
- kernels 
- users
- users achievements


## Users summary

In [ ]:
resumetable(users)

Here we can see informations about all Registrations on Kaggle. 

## Kernels summary

In [ ]:
resumetable(kernels)

Cool! <br>
The total of AuthorUserId is 100,631 that is the same number that are shown in the Kernels Rank profile. 

## Users Achievement

In [ ]:
resumetable(users_achi)

Cool! <br>At Users Achievement table we have the total of registrations multiplied by 3; <br>
This table shows the tier of each Kaggle category and the total medals for each one, date of achievement and points for each category type. <br>
We will explore it furthuer later. 

# Kaggle is a Trend?

Let's explore the total of registrations for each day since the launch of the platform.
What we will see:
- Count
- Mean
- Cumulative total Registers

Them try to analyze and understand the patterns, peaks, differences and so on

In [ ]:
def date_time_col(df, col, form='%m/%d/%Y'):
    df[col] = pd.to_datetime(df[col], format=form)
    return df[col]

users['RegisterDate'] = date_time_col(users, 'RegisterDate')

In [ ]:
# Calling the function to transform the date column in datetime pandas object

#seting some static color options
color_op = ['#5527A0', '#BB93D7', '#834CF7', '#6C941E', '#93EAEA', '#7425FF', '#F2098A', '#7E87AC', 
            '#EBE36F', '#7FD394', '#49C35D', '#3058EE', '#44FDCF', '#A38F85', '#C4CEE0', '#B63A05', 
            '#4856BF', '#F0DB1B', '#9FDBD9', '#B123AC']


dates_temp = users.groupby(users['RegisterDate'].dt.date)['UserId'].count().reset_index()
# renaming the columns to apropriate names

# creating the first trace with the necessary parameters
trace = go.Scatter(x=dates_temp['RegisterDate'], y=dates_temp.UserId,
                    opacity = 0.8, line = dict(color = color_op[7]), name= 'Registrations')

tmp_count = users.groupby(users['RegisterDate'])['UserId'].count().reset_index()
tmp_count['cumsum'] = tmp_count['UserId'].cumsum()
tmp_count['cumsum'] = tmp_count['cumsum'] 

# using the new dates_temp_sum we will create the second trace
trace1 = go.Scatter(x=tmp_count.RegisterDate, line = dict(color = color_op[1]), 
                    name="CumSum Registers",
                    y=tmp_count['cumsum'], opacity = 0.8, yaxis='y2')

tmp_mean = users.groupby(users['RegisterDate'].dt.date)['UserId'].count().reset_index()
tmp_mean['mean'] = round(tmp_mean['UserId'].rolling(90).mean())

# using the new dates_temp_sum we will create the second trace
trace2 = go.Scatter(x=tmp_mean.RegisterDate, line = dict(color = color_op[3]), 
                    name="MovAverage 90 days",
                    y=tmp_mean['mean'], opacity = 0.8)


layout = dict(
    title= "Kaggle Users Registration by Dates",
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=3, label='3m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(count=12, label='12m', step='month', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible = True),
        type='date' ),
    yaxis=dict(title='Total Registrations by Day'),
    yaxis2=dict(overlaying='y',
                anchor='x', side='right',
                zeroline=False, showgrid=False,
                title='Cumulative total subscribers'),
    annotations=[
        go.layout.Annotation(
            x='2017-06-11',
            y=1001168,
            xref="x",
            yref="y2",
            text="2017-06-11<br>1 Million day",
            showarrow=True,
            arrowhead=3,
            ax=-60,
            ay=-60
        ),
        go.layout.Annotation(
            x='2018-08-26',
            y=2001571,
            xref="x",
            yref="y2",
            text="2018-08-26<br>2 Million day",
            showarrow=True,
            arrowhead=3,
            ax=-80,
            ay=-40),
        go.layout.Annotation(
            x='2019-05-19',
            y=3002331,
            xref="x",
            yref="y2",
            text="2019-05-19<br>3 Million day",
            showarrow=True,
            arrowhead=3,
            ax=-80,
            ay=-30),
                
    ]
)

# creating figure with the both traces and layout
fig = dict(data= [trace, trace1, trace2],
           layout=layout)

#rendering the graphs
iplot(fig) #it's an equivalent to plt.show()

This chart is amazing. It is enough to tell us a good history of Kaggle growth! <br>
It's interesting to note that:
- Firts peak was on November, 4 of 2011 with 808 user registers. 
- Second peak was on December, 26 of 2013 with 1590 user registers.
- Third peak on May, 23 of 2016 with 1728 registers isn't so evident because that epoch Kaggle already had almost 1000 registration by day;
- After 2017 we can note a clear improvement and consolidation of more than 1k of registers each day. 
- The peak at March, 9 of 2017 with 3931 registers I find that is the day that Google announced the Kaggle acquirement.
- After it we can see that the number of registrations are apparently cyclical, let's explore other informations of user registration. 

Summary: We can see a clearly tendency in Kaggle registers, the number of register is very consistent and achieves easy 3k people registers by day with the highest peak at Apr 2019 with 5764 new registers. <br>
Looking this timeseries of registers I bet that Kaggle will keep receiving so much new data scientists and enthusiasts to the platform. 

<i> OFF: An interesting that I discovered is that I joined at Kaggle 11 days before it gets 1M users! =D </i>

## The Million Days
One interesting thing that I see is that:
- Jun, 11 of 2017 was the day that Kaggle achieved 1 Million registers. 
- Aug, 26 of 2018 achieved 2 Million Registers.
- May, 19 of 2019 achieved 3 Million Registers.

Cool! We are more than 3 million people all over the world. =D

# Explorating Registration Dates details
As we can see a Seasonality, let's see if the data shows is  

In [ ]:
users['month_reg'] = users['RegisterDate'].dt.month
users['dow_reg'] = users['RegisterDate'].dt.dayofweek
users['day_reg'] = users['RegisterDate'].dt.day

In [ ]:
#snsusers['dow_reg'].value_counts()
total = len(users)
plt.figure(figsize=(15,19))

plt.subplot(311)
g = sns.countplot(x="day_reg", data=users, color='coral')
g.set_title("User Registers by DAY OF MONTH", fontsize=20)
g.set_ylabel("Count",fontsize= 17)
g.set_xlabel("Day of Registrations", fontsize=17)
sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
g.set_ylim(0, max(sizes) * 1.15)

plt.subplot(312)
g2 = sns.countplot(x="dow_reg", data=users, color='coral')
g2.set_title("User Registers by DAY OF WEEK", fontsize=20)
g2.set_ylabel("Count",fontsize= 17)
g2.set_xlabel("Day of the Week of Registrations", fontsize=17)
sizes=[]
for p in g2.patches:
    height = p.get_height()
    sizes.append(height)
    g2.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=14) 
g2.set_ylim(0, max(sizes) * 1.15)

plt.subplot(313)
g1 = sns.countplot(x="month_reg", data=users, color='coral')
g1.set_title("User Registers by MONTHS", fontsize=20)
g1.set_ylabel("Performance Tier Cats",fontsize= 17)
g1.set_xlabel("", fontsize=17)
sizes=[]
for p in g1.patches:
    height = p.get_height()
    sizes.append(height)
    g1.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=14) 
g1.set_ylim(0, max(sizes) * 1.15)

plt.subplots_adjust(hspace = 0.3)

plt.show()

We can't see a very clear pattern considering these date features, but something call my attention. <br>

Some findings about the charts:
 - In days: It's apparently normal... All days of month are very similar less the day 31 that make all sense. 
 - In days of Week: We can see that Saturday and Sunday have the smallest % of registration with 10.7 ~ 10.9%;
 - In Months: We can see that December and January are the months with less registrations. 
 
Now, Let's use the crosstab function to see if have any relation between months and week days

## Crosstab Analysis - Day of the Week by Months

In [ ]:
cross_heatmap(users, 
              ['dow_reg', 'month_reg'],
              normalize='columns')

Interesting. <br>
It gives us a table showing the ratio of each day of the week for each month.
- We can note that the weekends are consistently worst than other days but in May it is even worst.

## Performance Tier Distribution at Kaggle

We have total of 3.4M unique registers in Kaggle. <br>
Before we go further at the timeseries to see the registers by the time, Let's see the Performance Tier Distribution;


In [ ]:
tier_dict = {0:'Novice', 1:'Contributor', 2:'Expert',
             3:'Master', 4:'GranMaster', 5:'KaggleTeam'}

users['PerformanceTier'] = users['PerformanceTier'].map(tier_dict)

plt.figure(figsize=(15,6))

g = sns.countplot(x="PerformanceTier", data=users, color='coral',
                  order=['KaggleTeam', 'Novice', 'Contributor', 
                         'Expert', 'Master', 'GranMaster'])
g.set_title("Performance Tier Distribution", fontsize=20)
g.set_ylabel("Performance Tier Cats",fontsize= 17)
g.set_xlabel("", fontsize=17)
sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.4f}%'.format(height/total*100),
            ha="center", fontsize=14) 
g.set_ylim(0, max(sizes) * 1.15)

plt.show()


Cool. We can see that only a small part of all users have at least contributor Tier. <br>
Could us infer that 97.9% of Kaggle community is passive? We can't forget that have many people that visit the platform but don't have already registered.  

## How many time in mean to get the tiers?

In [ ]:
users_comb = users_achi.merge(users, on='UserId')
users_comb['PerformanceTier'] = users_comb['PerformanceTier'].map(tier_dict)
users_comb['Tier'] = users_comb['Tier'].map(tier_dict)

In [ ]:
users_comb['RegisterDate'] = date_time_col(users_comb, 'RegisterDate')
users_comb['TierAchievementDate'] = date_time_col(users_comb, 'TierAchievementDate')

users_comb['diffReg'] = (users_comb['TierAchievementDate'].dt.date - users_comb['RegisterDate'].dt.date).dt.days


In [ ]:

table_user_tier = users_comb[(users_comb['Tier'] != 'Novice') & 
                             (users_comb['RegisterDate'] >= '2011-03-01')].groupby(['Tier', 
                                                                                    'AchievementType'])['diffReg'].mean().reset_index()
tier_count = users_comb[(users_comb['Tier'] != 'Novice') &
                        (users_comb['RegisterDate'] >= '2011-03-01')].groupby(['Tier',
                                                                               'AchievementType'])['UserId'].count().reset_index()


plt.figure(figsize=(15,11))

plt.subplot(211)
g0 = sns.barplot(x='Tier', y='UserId', 
                hue='AchievementType', data=tier_count,
                order = ['Expert', 'Master', 'GranMaster'])
g0.set_title("Distribution of each Category", fontsize=20)
g0.set_xlabel("Kaggle Category's", fontsize=17)
g0.set_ylabel("Count total", fontsize=17)
sizes=[]
for p in g0.patches:
    height = p.get_height()
    sizes.append(height)
    g0.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.0f}'.format(height),
            ha="center", fontsize=14) 
g0.set_ylim(0, max(sizes) * 1.15)

plt.subplot(212)
g = sns.barplot(x='Tier', y='diffReg', 
                hue='AchievementType', data=table_user_tier,
                order = ['Expert', 'Master', 'GranMaster'])
g.set_title("Time to take different Tiers (IN DAYS)", fontsize=20)
g.set_xlabel("Kaggle Category's", fontsize=17)
g.set_ylabel("Mean of Days", fontsize=17)

sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.0f}'.format(height),
            ha="center", fontsize=14) 
g.set_ylim(0, max(sizes) * 1.15)

plt.subplots_adjust(hspace = 0.3)

plt.show()


Cool! <br>
As I said I don't know exactly if it's really correct. but show us interesting and similar patterns. <br>
Only in Gran Master Tier we can see a clear difference in mean days.
<br>
I'm trying to do a correctly handling to avoid any type of leakage.<br>
We can see that when the time to a person do all things to become a Contributor is in median <br><br>

How can I be sure that it is correctly?! The Grandmaster of scripts seem to be wrong but I think it is because the outlier guys like Chris Deotte, MH Bahamani and nananshi that have rushed very fast. <br>I will investigate it further.  

# Kernels
What do we will analyze here?
- Total of Kernels created through the time
- How many kernels are created by month? Does this quantity is growing through time?

In [ ]:
users_comb = pd.merge(kernels, users, left_on='AuthorUserId', right_on='UserId', how='left')

In [ ]:
users_comb['CreationDate'] = date_time_col(users_comb, 'CreationDate',
                                           form='%m/%d/%Y %H:%M:%S')

users_comb['MadePublicDate'] = date_time_col(users_comb, 'MadePublicDate',
                                             form='%m/%d/%Y')
users_comb['RegisterDate'] = users_comb['RegisterDate'].dt.date
users_comb['CreationDate'] = users_comb['CreationDate'].dt.date
users_comb['MadePublicDate'] = users_comb['MadePublicDate'].dt.date

# To get votes by date
kernel_versions['CreationDate'] = date_time_col(kernel_versions, 'CreationDate',
                                                  form='%m/%d/%Y %H:%M:%S')

kernel_versions['CreationDate'] = kernel_versions['CreationDate'].dt.date

## Ploting Historical Kernels Creation
- Let's see the patterns of Kernel Creations by Dates

In [ ]:
# Calling the function to transform the date column in datetime pandas object

#seting some static color options
color_op = ['#5527A0', '#BB93D7', '#834CF7', '#6C941E', '#93EAEA', '#7425FF', '#F2098A', '#7E87AC', 
            '#EBE36F', '#7FD394', '#49C35D', '#3058EE', '#44FDCF', '#A38F85', '#C4CEE0', '#B63A05', 
            '#4856BF', '#F0DB1B', '#9FDBD9', '#B123AC']


tmp_kern = users_comb.groupby('CreationDate')['KernelId'].nunique().reset_index()
# renaming the columns to apropriate names

# creating the first trace with the necessary parameters
trace = go.Scatter(x=tmp_kern['CreationDate'], y=tmp_kern.KernelId,
                    opacity = 0.8, line = dict(color = color_op[7]), 
                   name= 'Daily Creations')


tmp_kern['cumsum'] = tmp_kern['KernelId'].cumsum()

# using the new dates_temp_sum we will create the second trace
trace1 = go.Scatter(x=tmp_kern.CreationDate, line = dict(color = color_op[1]), 
                    name="Total Sum Kernels",
                    y=tmp_kern['cumsum'], opacity = 0.8, yaxis='y2')

tmp_kern['mean'] = round(tmp_kern['KernelId'].rolling(30).mean())

# using the new dates_temp_sum we will create the second trace
trace2 = go.Scatter(x=tmp_kern.CreationDate, line = dict(color = color_op[3]), 
                    name="Mov.Average 30 Days", 
                    y=tmp_kern['mean'], opacity = 0.8)


##tmp_votes = kernel_versions.groupby('CreationDate')['TotalVotes_version'].sum().reset_index()

##trace3 = go.Scatter(x=tmp_votes.CreationDate, line = dict(color = color_op[6]), 
##                    name="Mov.Average 3fds0 Days",
##                    y=tmp_votes['TotalVotes_version'], opacity = 0.8)


layout = dict(
    title= "Kernel Informations by Date",
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=3, label='3m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(count=12, label='12m', step='month', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible = True),
        type='date' ),
    yaxis=dict(title='Kernel Creation by Day'),
    yaxis2=dict(overlaying='y',
                anchor='x', side='right',
                zeroline=False, showgrid=False,
                title='Cumulative Total Kernels'),

    annotations=[
            go.layout.Annotation(
                x='2017-06-11',
                y=43,
                xref="x",
                yref="y",
                text="NOTE<br>It's 1 million day. <br>what happened?",
                showarrow=True,
                arrowhead=3,
                ax=51,
                ay=-50
            )

        ]
)

# creating figure with the both traces and layout
fig = dict(data= [trace, trace1, trace2, #trace3
                 ],
           layout=layout)

#rendering the graphs
iplot(fig) 

Hey, we have a very important thing happening on the one Million day;  <br>

In my research to better understand this data I discovered that Kaggle only started the kernels in January, 2016; 

We can see some interesting peaks:
- Feb 26, 2016 was the first Peak. At this day was created 368 kernels;
- On May 3, 2017 the second Peak, It had 646 kernels created;
- The third and fourth peaks are very similar. These ocurred on Feb 13, 2018 and Mar 26, 2018 with 718 and 690 respectively;

It would be interesting if we can get informations about the One Million day. What happened this day?

# Competitions 
Now, we will explore the competitions:
- How many competitions we have? 
- Which are the distribution of paid and not paid competitions by the time?
- What is the distribution of Prizes in paid competitions? 

Let's investigate it further. 

In [ ]:
comp['RewardType'].fillna('None', inplace=True)

## Reward Type Distribution

In [ ]:
total = len(comp)

print(f'Total of registered competitions on Kaggle: {total}')

plt.figure(figsize=(14,5))

g = sns.countplot(x='RewardType', data=comp, color='coral')
g.set_title("Competition Reward Types", fontsize=22)
g.set_ylabel("Count", fontsize=17)
g.set_xlabel("Reward Type Name", fontsize=17)
sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=14) 
    
g.set_ylim(0, max(sizes) * 1.15)


plt.show()

Cool! <br>
It's a very interesting chart to understand Kaggle competitions profile. <br>

We can see that:
- Only 13.69% of all competitions has money prizes. 
- The None values (that is the most common value with 65.10%) we can consider as "Knowledge" competition. It's very interesting because sometimes we consider Kaggle a plataform of paid competitions, but the data shows us the truth. 

Now, that we have clealy this differentiation of paid or not comps, what about we plot the comps through the time and try understand how it happened;


In [ ]:
comp.loc[comp["RewardType"].isin(['Knowledge', 'Swag',
                                  'Jobs', 'Kudos']), 'MoneyComp'] = 0
comp['MoneyComp'].fillna(1, inplace=True)

comp['EnabledDate'] = date_time_col(comp, 'EnabledDate',
                                           form='%m/%d/%Y %H:%M:%S')
comp['EnabledDate'] = comp['EnabledDate'].dt.date

In [ ]:
tmp_comp = comp.groupby('EnabledDate')['Id'].count().reset_index()
money_comp = comp.loc[comp["MoneyComp"] == 1].groupby('EnabledDate')['Id'].count().reset_index()
knowledge_comp = comp.loc[comp["MoneyComp"] == 0].groupby('EnabledDate')['Id'].count().reset_index()

In [ ]:
# Calling the function to transform the date column in datetime pandas object

#seting some static color options
color_op = ['#5527A0', '#BB93D7', '#834CF7', '#6C941E', '#93EAEA', '#7425FF', '#F2098A', '#7E87AC', 
            '#EBE36F', '#7FD394', '#49C35D', '#3058EE', '#44FDCF', '#A38F85', '#C4CEE0', '#B63A05', 
            '#4856BF', '#F0DB1B', '#9FDBD9', '#B123AC']


#tmp_kern = comp.groupby('CreationDate')['KernelId'].nunique().reset_index()
# renaming the columns to apropriate names

# creating the first trace with the necessary parameters
trace = go.Scatter(x=money_comp['EnabledDate'], y=money_comp.Id,
                   opacity = 0.8, line = dict(color = color_op[7]), 
                   name= 'Money Comps')


tmp_comp['cumsum'] = tmp_comp['Id'].cumsum()

# using the new dates_temp_sum we will create the second trace
trace1 = go.Scatter(x=tmp_comp.EnabledDate, line = dict(color = color_op[1]), 
                    name="Total cumulative",
                    y=tmp_comp['cumsum'], opacity = 0.8, yaxis='y2')

# using the new dates_temp_sum we will create the second trace
trace2 = go.Scatter(x=knowledge_comp.EnabledDate, line = dict(color = color_op[3]), 
                    name="Knowledge_comps", 
                    y=knowledge_comp['Id'], opacity = 0.8)


##tmp_votes = kernel_versions.groupby('CreationDate')['TotalVotes_version'].sum().reset_index()

##trace3 = go.Scatter(x=tmp_votes.CreationDate, line = dict(color = color_op[6]), 
##                    name="Mov.Average 3fds0 Days",
##                    y=tmp_votes['TotalVotes_version'], opacity = 0.8)


layout = dict(
    title= "Competitions Creation Distribution by Dates",
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=3, label='3m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(count=12, label='12m', step='month', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible = True),
        type='date' ),
    yaxis=dict(title='Total by the days'),
    yaxis2=dict(overlaying='y',
                anchor='x', side='right',
                zeroline=False, showgrid=False,
                title='Cumulative Total Comps')
)

# creating figure with the both traces and layout
fig = dict(data= [trace, trace1, trace2, #trace3
                 ],
           layout=layout)

#rendering the graphs
iplot(fig) 

Cool!!! 
It's a good chart to understand the pattern of competitions on Kaggle and their distribution throught the time

We can note that something has changed after September, 2017.... Maybe new politician by Google or what? <br>
To keep at it, I will take a look at prizes distributions.

# Votes in Kernels - Total unique users votes 
The idea of it is take all votes and combine the tables until we get the total of votes per unique users to Kagglers. 

- The objective is understand the distribution of unique users that have voted in a other user
- We can get an index of purity in the kernel votes or detect posible frauds?
- How is the result of total kernels and total unique votes received? 


In [ ]:
user_votes = pd.merge(kernel_votes, users[['UserName','DisplayName', 
                                           'RegisterDate','UserId',
                                           'PerformanceTier' ]],
                      left_on='UserId', right_on='UserId', how='left')

kerneld_users_id = kernel_versions[['KVersId', 'KernelId', 'AuthorUserId']]
user_votes_2 = pd.merge(user_votes[['UserId', 'KernelVersionId',
                                    'UserName','DisplayName', 'KernVotId',
                                    'PerformanceTier']], 
                      kerneld_users_id,
                      left_on='KernelVersionId', right_on='KVersId', how='left')

user_votes_23 = pd.merge(user_votes_2, users[['UserName','DisplayName', 
                                           'RegisterDate','UserId',
                                           'PerformanceTier' ]], 
                         left_on='AuthorUserId', right_on='UserId', how='left')

In [ ]:
tmp = user_votes_23[user_votes_23['PerformanceTier_y'] != 'KaggleTeam'].groupby(["DisplayName_y",'PerformanceTier_y']).agg({'UserName_x': 'nunique',
                                                                                                      'KernVotId': 'nunique',
                                                                                                      'KernelId': 'nunique'}).reset_index()



In [ ]:
tmp.rename(columns={'DisplayName_y': 'User_Name',
                    'UserName_x':'Total_uniques',
                    'KernVotId': 'Total_Votes',
                    'PerformanceTier_y': 'Tier',
                    'KernelId': 'Total_Kernels'}, inplace=True)

Cool! Now, we have:

- Total votes by Users
- Total Unique votes by Users
- Total kernels by User

Using the cols of aggregation, we can also create new metrics to better understand the patterns:

mean_unique_ratio = Unique_user_votes / total votes <br>
kernel_votes_mean = Total_Votes / Total_Kernels <br>
kernel_unique_votes_mean = Unique_user_votes / Total_Kernels <br>



In [ ]:
tmp['vote_ratio'] = round(tmp['Total_Votes'] / tmp['Total_uniques'], 2)
tmp_uniq_vot = tmp[#(tmp.vote_ratio != np.inf) &
                     #(tmp.Total_uniques > 10) & 
                     (tmp.Tier.isin(['Expert', 'Master', 'GranMaster']))].sort_values('Total_uniques', ascending=False)[:30]
tmp_kernl_tot = tmp[#(tmp.vote_ratio != np.inf) &
                    # (tmp.Total_uniques > 10) & 
                     (tmp.Tier.isin(['Expert', 'Master', 'GranMaster']))].sort_values('Total_Votes', ascending=False)[:30]
tmp_kernl = tmp[#(tmp.vote_ratio != np.inf) &
                #     (tmp.Total_uniques > 10) & 
                     (tmp.Tier.isin(['Expert', 'Master', 'GranMaster']))].sort_values('Total_Kernels', ascending=False)[:30]
tmp_vote_ratio = tmp[(tmp.vote_ratio != np.inf) &
                     (tmp.Total_uniques > 10) & 
                     (tmp.Tier.isin(['Expert', 'Master', 'GranMaster']))].sort_values('vote_ratio',
                                                                                       ascending=False).head(30)

# Total votes from unique Users

In [ ]:
#snsusers['dow_reg'].value_counts()
plt.figure(figsize=(14,21))

plt.subplot(311)
g = sns.barplot(x="User_Name", y='Total_uniques',  
                data=tmp_uniq_vot,  color='sandybrown',
                   order=list(tmp_uniq_vot['User_Name'].values))
g.set_title("TOTAL UNIQUE VOTES \nTOP 30 USERS WITH MORE VOTE FROM UNIQUE USERS", fontsize=20)
g.set_ylabel("Count ",fontsize= 17)
g.set_xlabel("", fontsize=17)
sizes=[]
for p in g.patches:
    height = p.get_height()
    sizes.append(height)
g.set_ylim(0, max(sizes) * 1.15)
g.set_xticklabels(g.get_xticklabels(),
                  rotation=90)
gt = g.twinx()
gt = sns.pointplot(x='User_Name', y='vote_ratio', data=tmp_uniq_vot, color='green',
                   order=list(tmp_uniq_vot['User_Name'].values),
                   # color='black',
                   legend=False)
gt.set_ylim(tmp_uniq_vot['vote_ratio'].min()-.3,tmp_uniq_vot['vote_ratio'].max()*1.1)
gt.set_ylabel("Mean of votes by User", fontsize=16)

plt.subplot(312)
g1 = sns.barplot(x="User_Name", y='Total_Votes',  data=tmp_kernl_tot,  color='sandybrown')
g1.set_title("TOTAL VOTES \nTOP 30 USERS WITH MORE VOTES", fontsize=20)
g1.set_ylabel("Count ",fontsize= 17)
g1.set_xlabel(" ", fontsize=17)
sizes=[]
for p in g1.patches:
    height = p.get_height()
    sizes.append(height)

g1.set_ylim(0, max(sizes) * 1.15)
g1.set_xticklabels(g1.get_xticklabels(),
                  rotation=90)
gt = g1.twinx()
gt = sns.pointplot(x='User_Name', y='vote_ratio', data=tmp_kernl_tot,
                   order=list(tmp_kernl_tot['User_Name'].values),
                   color='green', legend=False)
gt.set_ylim(tmp_kernl_tot['vote_ratio'].min()-.3,
            tmp_kernl_tot['vote_ratio'].max()*1.1)
gt.set_ylabel("Mean of votes by User", fontsize=16)


plt.subplot(313)
g2 = sns.barplot(x="User_Name", y='vote_ratio',  
                 data=tmp_vote_ratio,  color='sandybrown')
g2.set_title("UNIQUE USERS VOTE RATIO \nTOP 30 Users with the highest mean vote from unique Users", fontsize=20)
g2.set_ylabel("Count ",fontsize= 17)
g2.set_xlabel("TOP 30 Users", fontsize=17)
sizes=[]
for p in g2.patches:
    height = p.get_height()
    sizes.append(height)
    g2.text(p.get_x()+p.get_width()/2.,
            height + .05,
            f'{height}',
            ha="center", fontsize=11)

g2.set_ylim(0, max(sizes) * 1.15)
g2.set_xticklabels(g2.get_xticklabels(),
                  rotation=90)

plt.subplots_adjust(hspace = 0.8)

plt.show()

Nice!!! Some interesting things to note:<br>
### Note: In this charts, I'm considering only Experts, Masters and Grandmasters. 
First Chart: 
- We can see that the "Top unique votes" are SRK with more than 7k votes from unique users. 
- The second and third (Anistropic and DATAI) are very close from each other.
- We can see interestingly that Andrew (top 1) and Pedro Marcelino (The king of titanic) are almost in a draw;

In the top 30 users with more votes from unique users, we have:
- 3 Contributors
- 2 Experts
- 14 master 
- 11 grandmaster

We can note that in this chart the highest ratio total_votes/unique_votes:<Br>
- 1.92 of Bojan Tunguz 
- 1.89 of DATAI 
- 1.76 Chris

So in mean, Bojan received 1.92 votes from each user. 
____________________
Second Chart:<br>
We have 4 guys with more than 8k votes
- DATAI have almost 12k votes in total. 
- SRK few more than 10k 
- Andrew with that is the 1º rank are the third with more votes ~ 9.5k

This time, we can see that the highest values of the ratio of unique users are: Yury Kashnitsky and NowYSM with ~2.5 votes for each user
___________________________

Third Chart:<br>
- We can see that many users have values more than 3 in mean of votes;
- I noted a common pattern, many of them have a high number of votes but a small value of forks.
- In general the ratio of Upvotes / Forks is higher than 2 ~ 3;


So, the top isn't necessary the guy with more unique votes or with highest diversity of votes.

The next object is to get the total kernels and divide by the total of unique user votes... I find that it will give us interesting insights. 

# TOP users with more Kernels
- Let's explore the users with more kernels

In [ ]:
plt.figure(figsize=(14,6))
g2 = sns.barplot(x="User_Name", y='Total_Kernels',  
                 data=tmp_kernl,  color='sandybrown',
                   order=list(tmp_kernl['User_Name'].values))
g2.set_title("KERNELS  <br>TOP 30 USERS WITH MORE KERNELS", fontsize=20)
g2.set_ylabel("Count ",fontsize= 17)
g2.set_xlabel("TOP 30 Users", fontsize=17)
sizes=[]
for p in g2.patches:
    height = p.get_height()
    sizes.append(height)
    g2.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.0f}'.format(height),
            ha="center", fontsize=11)
g2.set_ylim(0, max(sizes) * 1.15)
g2.set_xticklabels(g2.get_xticklabels(),
                  rotation=90)
gt = g2.twinx()
gt = sns.pointplot(x='User_Name', y='vote_ratio', data=tmp_kernl, color='green',
                   order=list(tmp_kernl['User_Name'].values),
                   # color='black',
                   legend=False)
gt.set_ylim(tmp_kernl['vote_ratio'].min()-.3,tmp_kernl['vote_ratio'].max()*1.1)
gt.set_ylabel("Mean of votes by User", fontsize=16)

plt.show()

WTF! Kevin have a lot of Kernels... But we have a problem here. When I go to her profile, I can see Kernels(412) where are the other kernels of him? Maybe it's hidden... Someone knows about that?
<br>
We can note that many beasts of DS have more than 100 kernels; 
- 

# I'm working at this Kernel, so upvote the kernel and stay tuned
- Please, DON'T FORGET THE VOTE, IT'S IMPORTANT TO ME GET THE GRANDMASTER TIER!!!! 